In [1]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
import pickle
import os
from tqdm.notebook import tqdm
import statsmodels.formula.api as sm
from scipy.stats import t 
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
sp = "sp500" # sp500 or sp600 run this code once with sp = "sp500" , and once with sp = "sp600"

In [3]:
df_path = "data/earning_call_dataframe/df_final_"+sp 
# df_path = "data/df_final"
with open(df_path,"rb") as file :
    df_final = pickle.load(file)

In [4]:
display(df_final)

,id,date,ticker,text,earning_call_quarter
0,4264085,2019-05-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q1
1,4285738,2019-08-14,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q2
2,4309111,2019-11-26,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q3
3,4325101,2020-02-18,A,Question-and-Answer Session\n \nOperator\n \n[...,2019Q4
4,4349475,2020-05-21,A,Question-and-Answer Session\n \nOperator\n \n[...,2020Q1
...,...,...,...,...,...
1855,4422752,2021-04-30,DRE,Question-and-Answer Session\n \nOperator\n \n[...,2021Q1
1856,4442846,2021-07-29,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q2
1857,4463265,2021-10-28,DRE,Question-and-Answer Session\n \nOperator\n \nT...,2021Q3
1858,4482287,2022-01-27,DRE,Question-and-Answer Session \n \nOperator\n \n...,2021Q4


# Import datas from yahoo finance

In [5]:
# Get the month from which we import stock returns :
    # Q1 : from january (1) to march (3)
    # Q2 : from april (4) to june (6)
    # Q3 : from july (7) to septmber (9)
    # Q4 : from october (10) to december (12)

#df_final['earning_call_quarter'] = pd.to_datetime(df_final['earning_call_quarter'])
df_final['date'] = pd.to_datetime(df_final['date'])

# Quarters start 1 end
quarter_start = df_final['earning_call_quarter'].iloc[0].quarter
quarter_end = df_final['earning_call_quarter'].iloc[-1].quarter 

# Months start 1 end
month_start = quarter_start*2-1
month_end = quarter_end*3

# Day end : 30 or 31 ?
if month_end in [3,12]:
    day_end = 31
else:
    day_end = 30


In [6]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

# Take a margin of 1 year to calculate abnormal returns
start = datetime.datetime(df_final['earning_call_quarter'].iloc[0].year-1,month_start,2)
end   = datetime.datetime(df_final['earning_call_quarter'].iloc[-1].year+1,month_end,day_end)

# Create empty dataframe
stock_final = pd.DataFrame()

# Iterate over each symbol
list_tickers = list(set(list(df_final['ticker'])))
for i in list_tickers:  
    
    # print the symbol which is being downloaded
    print( str(list_tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['ticker']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None
        
display(stock_final)

0 : ANET,1 : BA,2 : BAC,3 : ABMD,4 : CF,5 : CLX,6 : BXP,7 : CHTR,8 : CHRW,9 : DHR,10 : D,11 : DLTR,12 : ANTM,13 : BDX,14 : ALB,15 : BRO,16 : CL,17 : CMG,18 : DLR,19 : AAL,20 : CZR,21 : AZO,22 : AVY,23 : CHD,24 : CI,25 : DIS,26 : A,27 : ADSK,28 : C,29 : ABT,30 : ADP,31 : BBWI,32 : AMP,33 : ADM,34 : AMAT,35 : DHI,36 : AIZ,37 : APH,38 : DE,39 : AFL,40 : CME,41 : CB,42 : DRE,43 : AIG,44 : CDNS,45 : ABBV,46 : ALK,47 : BIIB,48 : BLK,49 : CDAY,50 : CINF,51 : CMI,52 : CSX,53 : CPT,54 : AOS,55 : CCI,56 : CVS,57 : AKAM,58 : DAL,59 : BALL,60 : APD,61 : ALGN,62 : ATO,63 : ALL,64 : CMA,65 : CDW,66 : BR,67 : CAH,68 : CBOE,69 : DD,70 : CE,71 : BSX,72 : BKNG,73 : CAT,74 : AXP,75 : BBY,76 : APA,77 : DOV,78 : COF,79 : APTV,80 : AVB,81 : BKR,82 : ARE,83 : AAPL,84 : AES,85 : CBRE,86 : CTSH,87 : AEE,88 : AMZN,89 : BK,90 : BMY,91 : CMS,92 : DFS,93 : BEN,94 : DG,95 : AME,96 : ANSS,97 : DGX,98 : AON,99 : AEP,

,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2018-01-02,58.895000,59.657501,56.689999,57.757500,57.757500,3488800,ANET
2018-01-03,58.000000,58.937500,57.889999,58.900002,58.900002,2234400,ANET
2018-01-04,59.062500,59.457500,57.852501,58.172501,58.172501,3348000,ANET
2018-01-05,58.612499,59.615002,58.605000,59.192501,59.192501,2021600,ANET
2018-01-08,59.810001,62.987499,59.627499,62.764999,62.764999,5578800,ANET
...,...,...,...,...,...,...,...
2022-06-10,97.440002,99.160004,97.010002,98.260002,98.260002,3003000,AEP
2022-06-13,97.099998,97.580002,92.610001,93.010002,93.010002,4241100,AEP
2022-06-14,93.120003,93.800003,89.360001,90.440002,90.440002,3482800,AEP


In [7]:
# Drop useless columns
stock_final = stock_final.drop(columns = ['Open','High','Low','Close','Volume'])
stock_final = stock_final.reset_index()
stock_final = stock_final.groupby(['ticker','Date']).last()

# Calculate stock returns
stock_final['stock_return'] = stock_final['Adj Close'].pct_change()

# Remove first date of each stock
stock_final = stock_final.drop(stock_final.groupby('ticker',as_index=False).nth(0).index)
stock_final = stock_final.reset_index()
stock_final = stock_final.drop(columns = ['Adj Close'])
display(stock_final)

,ticker,Date,stock_return
0,A,2018-01-03,0.025444
1,A,2018-01-04,-0.007501
2,A,2018-01-05,0.015988
3,A,2018-01-08,0.002146
4,A,2018-01-09,0.024554
...,...,...,...
112116,DRE,2022-06-10,-0.038625
112117,DRE,2022-06-13,0.009843
112118,DRE,2022-06-14,0.018500
112119,DRE,2022-06-15,0.048437


In [8]:
# Market datas

# Market ticker
if sp == "sp500":
    sp_ticker = '^SPY' 
elif sp == "sp600":
    sp_ticker = '^SP600' 

# Import market datas
mkt =  yf.download(sp_ticker, start=start, end=end, progress=False)
# Compute market return
mkt['market_return'] = mkt['Adj Close'].pct_change()
mkt = mkt.drop(columns = ['Adj Close','Open','High','Low','Close','Volume'])
# Drop first row
mkt = mkt[1:]

# Abnormal returns

In [9]:
# Parameter
number_days_end = 30
number_days_start = 30

# Initialisation of final dataframe
df_plot = pd.DataFrame(data={'normalized_date': np.arange(-number_days_start , number_days_end+1, 1, dtype=int)})

# Vector of normalized_date
normalized_date = np.concatenate((np.arange(-number_days_start , 0, 1, dtype=int),[0], np.arange(0, number_days_end +1, 1, dtype=int)))

# Get list of different tickers of the dataframe
list_tickers = list(set(list(df_final['ticker'])))

In [10]:
for symbol in tqdm(list_tickers):

    # Calculate abnormal returns of each stock
    df_stock = stock_final[stock_final['ticker'] == symbol]
    df_stock = df_stock.set_index('Date')
    df_stock = pd.merge(df_stock, mkt, left_index=True, right_index=True)
    df_stock['abnormal_return'] = df_stock['stock_return']-df_stock['market_return']
    df_stock = df_stock.drop(columns = ['stock_return', 'market_return'])
    df_stock['id'] = 0*len(df_stock)
    first_column = df_stock.pop('id')
    df_stock.insert(0, 'id', first_column)

    # Dataframe with publication date of earning transcript
    df_final_stock = df_final[df_final['ticker'] == symbol]
    df_final_stock= df_final_stock.set_index('date')
    df_final_stock = df_final_stock.drop(columns = ['earning_call_quarter','text'])
    df_final_stock['abnormal_return']=0*len(df_final_stock)

    # Concatenate 2 previous dataframe and sort by date
    result = pd.concat([df_stock,df_final_stock])
    result = result.reset_index()
    result = result.rename(columns={'index': 'date'})
    result = result.sort_values(by=['date', 'id'])
    result = result.reset_index()

    # Get the indices of parution dates
    indices = result.loc[result['id'] != 0].index.values
    indices = indices[:-1] # remove most recent earning call

    for i in indices: # loop for each earning call 
        i = i.item() #convert to int
        if result['date'][i] != result['date'][i-1]:   # weekend : if an article is published during the weekend, the date 0 if there is no return at parution date, take the 
            first_df=[]
            first_df = result[i-(number_days_start) :i+(number_days_end+2)]
            first_df['normalized_date']=normalized_date
            first_df=first_df[first_df.index != i]
            first_df['cum_ab_return']=first_df['abnormal_return'].cumsum(axis = 0)
            column_title= result['id'].loc[i]
            df_plot[column_title]=first_df['cum_ab_return'].values
        else : 
            first_df=[]
            first_df = result[i-(number_days_start+1) :i+(number_days_end+1)]
            first_df['normalized_date']=normalized_date
            first_df=first_df[first_df.index != i]
            first_df['cum_ab_return']=first_df['abnormal_return'].cumsum(axis = 0)
            column_title= result['id'].loc[i]
            df_plot[column_title]=first_df['cum_ab_return'].values

  0%|          | 0/100 [00:00<?, ?it/s]

KeyError: -1

In [ ]:
col = list(df_plot)
col.remove('normalized_date')
title = sp+" Cumulative abnormal returns without deciles"
plot = df_plot.plot(x='normalized_date',y=col,figsize=(10,6))
plot.get_legend().remove()
plt.axvline(x=0,color="black",linestyle="--")
plt.xlabel("Normalized date")
plt.ylabel("Cumulative abnormal returns")
plt.title(title)
plt.savefig('./data/abnormal_graphs/'+title)

In [ ]:
#score_method = 'FiGAS' #figas or finbert
#content = 'text'  # 'text' or 'corpus'
#score = columns of csv file to take for the score

In [ ]:
def plot_with_deciles(col,score_method,content,score):
    file = "data/sentiment_score/"+score_method+"/"+score_method+ "_" + sp +"_by_"+content +"_final.csv"  #### CHANGE
    merged = []

    # Create score dataframe
    sentiment_text = pd.read_csv(file)  
    if score_method =='FiGAS':
        sentiment_text = sentiment_text.drop(columns = ['Unnamed: 0'])
    if score_method =='FinBert':
        sentiment_text = sentiment_text.rename(columns={"ID": "id"})
    sentiment_text['id'] = sentiment_text['id'].map(str)

    # Get the ids of dataframe col
    col_id = pd.DataFrame(col, columns =['id'])

    # Merge
    merged = pd.merge(col_id, sentiment_text, on='id')
    merged = merged.sort_values(by=[score])

    # Create deciles
    merged['decile'] = 1 + merged[score].transform(lambda y:pd.qcut(y,10,labels=False))

    # Compute abnormal returns for each decile
    df_decile_plot = []
    df_decile_plot = pd.DataFrame(data={'normalized_date': np.arange(-number_days_start , number_days_end+1, 1, dtype=int)})

    for i in range(1,11):
        dec1 = merged.loc[merged['decile'].isin([i])]
        long1 = dec1.id
        long1 = long1.to_list()

        df_mean = df_plot[long1].mean(axis=1)
        df_decile_plot['decile'+str(i)]=df_mean

    # Plot
    col = list(df_decile_plot)
    col.remove('normalized_date')
    title = sp+" Cumulative abnormal returns based on " +score_method+ " score (using "+ content + " and "+ score+")"
    plot = df_decile_plot.plot(x='normalized_date',y=col,figsize=(10,6))
    plt.axvline(x=0,color="black",linestyle="--")
    plt.legend(bbox_to_anchor=(1.0,1.0))
    plt.xlabel("Normalized date")
    plt.ylabel("Cumulative abnorman returns")
    if sp == "sp600" : 
        y_limit = [-0.03, 0.065];
    elif sp == "sp500" :
        y_limit = [-0.03, 0.04];
    plt.ylim(y_limit)
    plt.title(title)
    plt.savefig('./data/abnormal_graphs/'+title, bbox_inches='tight')

# Figas

## sent_score

In [ ]:
plot_with_deciles(col,'FiGAS','text','sent_score')
plot_with_deciles(col,'FiGAS','corpus','sent_score')

## sent_score_rough

In [ ]:
plot_with_deciles(col,'FiGAS','text','sent_score_rough')
plot_with_deciles(col,'FiGAS','corpus','sent_score_rough')

# FinBert

In [ ]:
plot_with_deciles(col,'FinBert','text','Difference ratio')